In [1]:
import pandas as pd

df = pd.read_csv("../raw_data/en.openfoodfacts.org.products.csv", delimiter='\t')

/tmp/ipykernel_7688/988412624.py:3: DtypeWarning: Columns (0,8,13,19,20,21,22,23,27,28,29,31,32,40,41,42,52,55,64,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../raw_data/en.openfoodfacts.org.products.csv", delimiter='\t')


Nous commencons par supprimer les colonnes avec plus de 50% de missing values

In [2]:
print('NaN rate per colum:')
for col in df.columns:
    sum = df[col].isna().sum()
    print(col + ': ' + str(sum/len(df)*100) + '%')
    if sum/len(df)*100 > 50:
        print('Dropping ' + col)
        df.drop(columns=col, inplace=True)

NaN rate per colum:
code: 0.0%
url: 0.0%
creator: 0.00019423529080422148%
created_t: 0.0%
created_datetime: 0.0%
last_modified_t: 0.0%
last_modified_datetime: 0.0%
product_name: 4.048688960345894%
abbreviated_product_name: 99.64945385892108%
Dropping abbreviated_product_name
generic_name: 93.99730401416365%
Dropping generic_name
quantity: 74.77296322446122%
Dropping quantity
packaging: 84.63054990924356%
Dropping packaging
packaging_tags: 84.6324922621516%
Dropping packaging_tags
packaging_text: 99.59793294803526%
Dropping packaging_text
brands: 48.86571446052605%
brands_tags: 48.86911357811512%
categories: 54.05796369548179%
Dropping categories
categories_tags: 54.05835216606341%
Dropping categories_tags
categories_en: 54.05835216606341%
Dropping categories_en
origins: 95.70074751451666%
Dropping origins
origins_tags: 95.7070601614678%
Dropping origins_tags
origins_en: 95.7070601614678%
Dropping origins_en
manufacturing_places: 93.75266466539573%
Dropping manufacturing_places
manufact

In [3]:
print(df.columns)

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name', 'brands',
       'brands_tags', 'countries', 'countries_tags', 'countries_en',
       'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en',
       'image_url', 'image_small_url', 'image_nutrition_url',
       'image_nutrition_small_url', 'energy-kcal_100g', 'energy_100g',
       'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g'],
      dtype='object')


Nous pouvons maintenant supprimer certaines colonnes qui n'apporteront rien pour le clustering:
- ```urlcreator, created_t, created_datetime, states, states_tags, last_modified_datetime, last_modified_t, image_url, image_small_url, image_nutrition_url, image_nutrition_small_url, states, states_en``` &rarr; Variables portant sur l'entrée du produit dans la bdd openfoodfacts
- ```product_name``` &rarr; A étudier lorsque la variable sera encodée pour voir les changements sur le modèle
- ```countries, countries_en``` &rarr; Redondant avec country_tags
- ```energy_100g``` &rarr; redondant avec ```energy-kcal_100g```
- ```brands``` &rarr; Nom de la marque.

In [4]:
columns = ['url', 'creator', 'created_t', 'created_datetime', 'states', 'states_tags', 'last_modified_datetime', 'last_modified_t',
'image_url', 'image_small_url', 'image_nutrition_url', 'image_nutrition_small_url', 'countries','countries_en',
'energy_100g', 'product_name', 'states', 'states_en', 'brands']

df.drop(columns=columns, inplace=True)
# to_drop = [col for col in columns if col in df.columns]
# df.drop(columns=to_drop, inplace=True)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059358 entries, 0 to 2059357
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   code                object 
 1   brands_tags         object 
 2   countries_tags      object 
 3   pnns_groups_1       object 
 4   pnns_groups_2       object 
 5   energy-kcal_100g    float64
 6   fat_100g            float64
 7   saturated-fat_100g  float64
 8   carbohydrates_100g  float64
 9   sugars_100g         float64
 10  proteins_100g       float64
 11  salt_100g           float64
 12  sodium_100g         float64
dtypes: float64(8), object(5)
memory usage: 204.3+ MB


Les colonnes restantes:

- ```pnns_groups_1``` et ```pnns_groups_2``` &rarr; Ces colonnes correspondent deja 
- ```fat_100g, saturated-fat_100g```, ```carbohydrates_100g```, ```sugars_100g```, ```proteins_100g```, ```salt_100g```, ```sodium_100g``` &rarr; Valeurs nutritionelles
- ```energy-kcal_100g``` &rarr; Les fabricants de produits alimentaires calculent cette valeur en additionnant où kcal/g de protéines, lipides et glucides contenus dans leur produit. [source](https://fr.wikipedia.org/wiki/Composition_nutritionnelle_des_aliments) &rarr; à vérifier si cette valeur n'est pas redondante avec les valeurs nutritionelles
- ```nutrition-score-fr_100g```
-  ```brands_tags``` &rarr; Nom de la marque sans caractères spéciaux. Un certain nombre de Nan mais une information qui est parfois utile lorsqu'elle est présente (Intuitivement par exemple nous savons qu'un produit de la marque Picard sera très probablement un produit surgelé). A encoder
- ```code``` &rarr; Cette variable correspond au code EAN13. Une partie du code correspond au pays et à l'entreprise emmetrice du produit [source](https://www.laintimes.com/explication-et-construction-des-codes-barres-suivant-la-norme-ean-13/)
A réencoder en 3 colonnes différentes, `Pays`, `Entreprise`, `Code Produit` et supprimer le dernier chiffre
- ```country_tags``` &rarr; Pays ou le produit est vendu